### 자기소개서 전문가 총평 감성분석

In [1]:
from konlpy.tag import Okt
import konlpy
import pandas as pd

In [2]:
df = pd.read_csv('essay.csv')

In [14]:
df.isnull().sum()

Unnamed: 0           0
company_id           0
company              0
year                 0
season               0
experience           0
position             0
score             1200
feedback          1202
content              0
views                0
position_broad       0
dtype: int64

In [19]:
df = df[df.feedback.notna()] #결측치 제외
df = df[['score' , 'feedback']]

In [21]:
df.groupby('score').size()

score
1.0     61
2.0    549
3.0    288
3.5      1
4.0    163
4.5      4
5.0     50
dtype: int64

4점 이상 : 217개 <br>
1점 : 61개 <br><br>

217 : 217 로 맞춰주기 위해서 2점은 156개만 추출

In [24]:
#데이터 수를 맞춰주기 위해  2점짜리 언더 샘플링 

two_point = df[(df['score'] == 2)].sample(n = 156, random_state = 0)
len(two_point)

156

In [37]:
# 1,2점  + 4 ,5 점으로 dataframe 만들기 

result = df[(df['score'] == 1) | (df['score'] >= 4) ]

result = pd.concat([result , two_point])

In [38]:
result.sort_values('score' , inplace= True)
result.reset_index(drop=True , inplace=True)

In [40]:
result['score'] = result['score'].map(lambda x: 0 if x <= 2 else 1)

In [42]:
result.groupby('score').size()

score
0    217
1    217
dtype: int64

In [47]:
#불용어 처리
stopwords = pd.read_json('stopwords-ko.json')
stopwordsList = list(stopwords[0]) + ['이다','하다']

In [55]:
def preprocessing(text):
    okt = Okt()
    raw_pos_tagged = okt.pos(text, norm=True,stem=True)
    
    cleanWords = []
    
    for word, tag in raw_pos_tagged: 
        if tag not in ["Josa", "Eomi", "Punctuation", "Foreign"]: 
            if (len(word) != 1) & (word not in stopwordsList): 
                cleanWords.append(word)
    
    new_text = ' '.join(cleanWords)
    
    return new_text

In [56]:
result['feedback'] = result['feedback'].map(lambda x : preprocessing(x))

In [59]:
result['feedback'][0:3].values

array(['개별 항목 목마 요구 내용 즉흥 작성 보이다 소개 소개 전체 구성 보다 지원 직무 관련 역량 강점 보여주다 전략 전혀 보이지 않다 내용 읽다 지원 인사 총무 직무 관련 되다 어떻다 지식 역량 경험 가지다 건지다 수가 없다 항목 요구 인지 이해 과정 선행 되어다 보이다 항목 자체 이해 항목 요구사항 관련 없다 내용 구성 되어다 부분 많이 띄다',
       '아무리 분야 관련 경험 준비 호감 주다 회사 회사 쓰다 가장 크다 실수 이유 불문 탈락 지원 대위 지원 내용 그대로 복사 활용 과정 회사 수정 않다 특히 항목 마지막 기계 쓸다 다짐 대위 회사 반복 나오다 평가 필요 없다 소개',
       '전반 아쉬움 많다 소개 보다 자다 개다 합격 이유 찾기 어렵다 자다 개다 없다 인사 담당자 호감 요인 인하다 합격 아니다 생각 소개 항목 의미 의도 자다 모르거나 쓰기 편하다 내용 쓸다 부분 보이다 목마 요구 사항 충족 않다 자다 개다 요구 사항 명확하다 이해 그게 맞다 작성 제일 중요하다'],
      dtype=object)

In [60]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from sklearn import datasets, model_selection, linear_model
from sklearn.pipeline import Pipeline

In [66]:
x_data = result.feedback.values
y_data = result.score.values

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=0)

In [67]:
print(len(X_train), np.bincount(y_train)) # train 데이터 크기 확인
print(len(X_test), np.bincount(y_test))

303 [149 154]
131 [68 63]


In [68]:
okt = Okt()

tfidf = TfidfVectorizer(lowercase=False, tokenizer = okt.morphs)
model = linear_model.LogisticRegression()
pipe = Pipeline([('vect', tfidf), ('clf', model)])
pipe.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(lowercase=False,
                                 tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x7fcf85256590>>)),
                ('clf', LogisticRegression())])

In [70]:
print('Accuracy: ', accuracy_score(pipe.predict(X_test), y_test))

Accuracy:  0.9389312977099237


In [80]:
# 총평 긍/부정 분류기

def sentiment_cls():
    feedback = input('총평을 입력하세요 : ')
    
    processed = preprocessing(feedback)
    prediction = pipe.predict([processed])
    
    if prediction == 0: # 부정
        print('\n' , np.max(pipe.predict_proba([processed])) * 100, '% 의 확률로 부정적인 총평입니다.')
    else : # 긍정
        print('\n' , np.max(pipe.predict_proba([processed])) * 100, '% 의 확률로 긍정적인 총평입니다.')

In [88]:
#별점 5점짜리 총평

sentiment_cls()

총평을 입력하세요 : 범한 지원동기와 입사 후 포부를 제외하고 나머지 부분은 수정할 필요가 없이 핵심 위주로 잘 쓴 자기소개서 입니다. 전체적으로 읽어보면 지원자는 지원하는 회사에 대해 많은 것을 알고 있다는 것을 알 수 있습니다. 즉, 지원회사에 대한 애정이 있고 시간을 두고 준비를 해 왔다는 느낌을 주고 있습니다.. 지원자의 가장 큰 강점은 글을 잘 쓴다는 것입니다. 한편의 보고서를 보는 것 같이 요구사항에 딱 맞아 떨어지는 소재와 내용으로 군더더기 없이 잘 작성하였고, 지원자가 경험한 사례 역시 신입지원자로서는 수준이 느껴지는 경험이었다는 점이 호감을 주고 있습니다. 면접에서 한 번 보고 싶은 지원자입니다.

 77.71792962496437 % 의 확률로 긍정적인 총평입니다.


In [89]:
#별정 1점짜리 총평

sentiment_cls()

총평을 입력하세요 : 항목의 요구사항에 완벽히 맞도록 작성하지 않은 부분이 대부분이라 아쉽습니다. 지원자의 강점을 직접적으로 묻는 항목에서는 자신의 직무강점이나 역량이 아닌 자신이 중요하게 생각하는 가치에 대해서 작성했고, 단점을 묻는 항목에서는 장단점을 작성하고 장점에 더 많은 분량을 할애하고 있습니다. 또한 회사에 입사하기 위한 노력을 묻는 항목에서는 직무선택동기를 작성하고, ‘차근차근’준비했다고 말하고 있지만 정확히 어떤 것들을 준비해왔다는 것인지 알수가 없습니다. 마지막으로 많지 않은 분량의 글에 매 항목마다 마지막 문장을 기계적인 다짐이나 포부로 작성하였습니다. 하지만 이런 글을 썼다고 해서 지원자에게 특별히 호감을 갖는 인사담당자는 많지 않습니다. 이런 기계적 다짐보다는 항목에서 요구하는 내용에 초점을 맞춰 작성해야 합니다.

 60.26296079572283 % 의 확률로 부정적인 총평입니다.


In [90]:
#반어법 감지 못함

sentiment_cls()

총평을 입력하세요 : 내가 써도 이거보단 잘 쓰겠다.

 61.82821738141294 % 의 확률로 긍정적인 총평입니다.


In [91]:
# 짧은 총평

sentiment_cls()

총평을 입력하세요 : 진짜 이건 무조건 합격임

 58.562255661673944 % 의 확률로 긍정적인 총평입니다.
